In [1]:
import pandas as pd
import subprocess

In [2]:
#PheWAS
catalog=pd.read_csv("phewascatalog_full.csv")
PD_snps_PheWAS=catalog[catalog["phewas phenotype"]=="Parkinson's disease"] #121 Parkinson SNPs
PD_snps_ids=PD_snps_PheWAS["snp"].tolist()
with open ("PheWAS_SNPIDs.txt","w") as PheWAS_SNPIDs:
    for ID in PD_snps_ids:
        PheWAS_SNPIDs.write(ID+"\n")

In [ ]:
#### Query Haploreg with PheWAS data ####

In [16]:
#Haploreg Query from browser, default values
Haploreg_queryPheWAS=pd.read_csv("haploreg_query_PHEWAS.txt",delimiter = '\t')
#keep column 6 and 25 (index 5 and 24). "SNP ID" and "Gene name"
Haploreg_queryPheWAS_map = Haploreg_queryPheWAS[['rsID','GENCODE_name']]
print ("initial size:",len(Haploreg_queryPheWAS_map))
#remove duplicates, keep first occurence
Haploreg_queryPheWAS_map_unique=Haploreg_queryPheWAS_map.drop_duplicates(keep="first",inplace=False)
print("Duplicates removed",len(Haploreg_queryPheWAS_map_unique))

initial size: 3977
Duplicates removed 3928


In [17]:
#remove NaN valued rows and shady values 
Haploreg_queryPheWAS_map_unique_nonan=Haploreg_queryPheWAS_map_unique.dropna(inplace=False)
print("NaN values removed",len(Haploreg_queryPheWAS_map_unique_nonan))

NaN values removed 3914


In [18]:
#make copy of mapping with legit name
Haploreg_PheWAS_final=Haploreg_queryPheWAS_map_unique_nonan
print (Haploreg_PheWAS_final)

             rsID GENCODE_name
0      rs13037879         UQCC
1      rs13038012         UQCC
2       rs8115394         UQCC
3       rs2425054         UQCC
4       rs8117705         UQCC
...           ...          ...
3972   rs10192369        RBMS1
3973    rs7420624        RBMS1
3974  rs143632448        RBMS1
3975    rs7250872       ATP8B3
3976   rs10418560       ATP8B3

[3914 rows x 2 columns]


In [ ]:
#### Query GTEx (gtex_v8) with PheWAS data ####

In [40]:
#access Gtex portal thru API "https://gtexportal.org/home/api-docs/index.html#!/association/singleTissueEqtl"
#fetch only brain tissue qtls
with open('Phewas_snp_gene_eqtl.csv', 'a') as outfile: # save in csv
    for snp in PD_snps_ids:
        command="curl -X GET --header \'Accept: text/html\' \'https://gtexportal.org/rest/v1/association/singleTissueEqtl?format=tsv&snpId="+snp+"&tissueSiteDetailId=Brain_Amygdala%2CBrain_Anterior_cingulate_cortex_BA24%2CBrain_Caudate_basal_ganglia%2CBrain_Cerebellar_Hemisphere%2CBrain_Cerebellum%2CBrain_Cortex%2CBrain_Frontal_Cortex_BA9%2CBrain_Hippocampus%2CBrain_Hypothalamus%2CBrain_Nucleus_accumbens_basal_ganglia%2CBrain_Putamen_basal_ganglia%2CBrain_Spinal_cord_cervical_c-1%2CBrain_Substantia_nigra&datasetId=gtex_v8\'"
        subprocess.call(command,shell=True,stdout=outfile)

In [41]:
eQTL_query_PheWAS=pd.read_csv("Phewas_snp_gene_eqtl.csv",sep="\t")
#keep only snpID and gene symbol column
eQTL_query_PheWAS=eQTL_query_PheWAS[["snpId","geneSymbolUpper"]]
print ("Initial size:",len(eQTL_query_PheWAS))
print (eQTL_query_PheWAS)

Initial size: 455
         snpId  geneSymbolUpper
0    rs6088792             GDF5
1    rs6088792          RPL36P4
2    rs6088792          RPL36P4
3    rs6088792             GDF5
4    rs6088792          RPL36P4
..         ...              ...
450      snpId  geneSymbolUpper
451  rs7250872           ATP8B3
452  rs7250872           ATP8B3
453  rs7250872           ATP8B3
454  rs7250872           ATP8B3

[455 rows x 2 columns]


In [42]:
#drop duplicates
eQTL_query_PheWAS=eQTL_query_PheWAS.drop_duplicates(keep="first",inplace=False)
print ("No duplicates:",len(eQTL_query_PheWAS))

No duplicates: 112


In [44]:
#drop shady row nr 5
eQTL_query_PheWAS=eQTL_query_PheWAS.drop(eQTL_query_PheWAS.index[5])
print("final size",len(eQTL_query_PheWAS))
print (eQTL_query_PheWAS)

final size 110
         snpId geneSymbolUpper
0    rs6088792            GDF5
1    rs6088792         RPL36P4
5    rs6088792           UQCC1
10   rs6088792        MAP1LC3A
13   rs6088792           MYH7B
..         ...             ...
434   rs987870         COL11A2
438   rs987870            RXRB
440  rs1494961         MRPS18C
441  rs1494961      SLC25A14P1
451  rs7250872          ATP8B3

[110 rows x 2 columns]


In [70]:
#### combine Haploreg and GTEx results and only consider eQTL SNPs, which are in Haploreg results
#
#Drop snps that are not in Haploreg results
haplo_ids=Haploreg_PheWAS_final.rsID.tolist()
eQTL_query_PheWAS_haplofilter=eQTL_query_PheWAS[eQTL_query_PheWAS["snpId"].isin(haplo_ids)]

In [74]:
#renaming eQTL_query_PheWAS_haplofilter column names
eQTL_query_PheWAS_haplofilter.columns=["rsID","GENCODE_name"]

In [78]:
#join Haploreg_PheWAS_final and eQTL_query_PheWAS_haplofilter
frames = [Haploreg_PheWAS_final,eQTL_query_PheWAS_haplofilter]
snps_gene_haploreg_gtex = pd.concat(frames, keys=['haplo', 'gtex'])

snps_gene_haploreg_gtex

rsID GENCODE_name
haplo 0    rs13037879         UQCC
      1    rs13038012         UQCC
      2     rs8115394         UQCC
      3     rs2425054         UQCC
      4     rs8117705         UQCC
...               ...          ...
gtex  434    rs987870      COL11A2
      438    rs987870         RXRB
      440   rs1494961      MRPS18C
      441   rs1494961   SLC25A14P1
      451   rs7250872       ATP8B3

[4007 rows x 2 columns]

In [79]:
#remove duplicates from joined dataframe
snps_gene_haploreg_gtex=snps_gene_haploreg_gtex.drop_duplicates(keep="first",inplace=False)

In [81]:
snps_gene_haploreg_gtex #done
snps_gene_haploreg_gtex.to_csv("snps_gene_haploreg_gtex.csv",index=False)

In [ ]:
#################################################################